In [12]:
from __future__ import print_function
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import os, sys
import torchvision
from torch.utils.tensorboard import SummaryWriter
import socket
from datetime import datetime
import scipy.io as sio   
from scipy.io import loadmat
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import getpass
print('__cuda available ',torch.cuda.is_available())
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Device name:', torch.cuda.get_device_name(0))

__cuda available  True
__Python VERSION: 3.6.10 (default, Apr 23 2020, 15:24:07) 
[GCC 8.3.0]
__pyTorch VERSION: 1.5.0+cu101
__CUDNN VERSION: 7603
__Number CUDA Devices: 4
__Device name: Quadro RTX 6000


In [13]:
user=getpass.getuser()
print(user)
if user=='eghbalhosseini':
    save_dir='/Users/eghbalhosseini/MyData/neural_manifolds/CNN_training_on_parition/'
    data_dir='/Users/eghbalhosseini/MyData/neural_manifolds/'
elif user=='ehoseini':
    save_dir='/om/user/ehoseini/MyData/neural_manifolds/CNN_training_on_parition/'
    data_dir='/om/user/ehoseini/MyData/neural_manifolds/'
#data_file='synthpartition_nobj_60000_nclass_10_nfeat_784_norm_1.mat'
data_file='synthtree_nobj_100000_nclass_50_nfeat_784_norm_1.mat'

ehoseini


In [14]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx % log_interval == 0) & (batch_idx!=0 & False):
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct = pred.eq(target.view_as(pred)).sum().item()
            # 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            n_iter=batch_idx+(epoch-1)*len(train_loader)
            writer.add_scalar('Loss/train', loss,n_iter )
            writer.add_scalar('Accuracy/train', 100. * correct / len(target), n_iter)
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): #don't save gradient
        for data, target in test_loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    n_iter=(epoch-1)
    writer.add_scalar('Loss/test', test_loss,n_iter )
    writer.add_scalar('Accuracy/test', 100. * correct / len(test_loader.dataset), n_iter)

In [15]:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')

access_rights = 0o755
try:
    os.mkdir(save_dir,access_rights)
except OSError:
    print('Creation of %s failed\n'%save_dir)

batch_size=32
test_batch_size=200 
train_set_size=60000
test_set_size=1000
epochs=100
no_cuda=False
save_epochs=False
momentum=0.2
lr=0.02
seed=1
gamma=0.7
log_interval=250
cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

train_spec={'batch_size':batch_size,
           'train_set_size':train_set_size,
           'num_epochs':epochs,
           'lr':lr,
           'log_interval':log_interval,
           'test_batch_size':test_batch_size
           }

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}


Creation of /om/user/ehoseini/MyData/neural_manifolds/CNN_training_on_parition/ failed



In [16]:
class partition_dataset(Dataset):
    def __init__(self, data_dir=data_dir):
        self.data_dir=data_dir
        self.dat , self.target=self.load_data()
        self.n_samples=self.dat.shape[0]
    def __len__(self):
        return self.n_samples
    def __getitem__(self, idx):
        item=np.expand_dims(self.dat[idx],axis=0)
        targ=np.squeeze(self.target[idx])
        return torch.tensor(item,dtype=torch.float), torch.tensor(targ,dtype=torch.long)
    def load_data(self):
        annot=loadmat(self.data_dir) 
        ops_struct=annot['ops_out']
        vals=ops_struct[0,0]
        dat=vals['data']
        self.adj=vals['Adjacency']
        dat_new=dat[:,range(28*28)]
        dat_new=np.reshape(dat_new,(-1,28,28))
        target=np.double(np.transpose(vals['class_id'])-1.0)
        return dat_new, target 

# load dataset 
partition_train_dataset=partition_dataset(data_dir=os.path.join(data_dir,data_file))

In [17]:


train_loader = torch.utils.data.DataLoader(
    dataset=partition_train_dataset,
    batch_size=32,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=partition_train_dataset,
    batch_size=1000,
    shuffle=True)


In [22]:
iteration=iter(test_loader)
epoch_data,epoch_target = next(iteration)
print('data shape',epoch_data.shape)


data shape torch.Size([1000, 1, 28, 28])


In [23]:
# Build the neural network, expand on top of nn.Module
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 50)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [24]:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
log_dir = os.path.join(save_dir,'runs', current_time + '_' + socket.gethostname())
writer = SummaryWriter(log_dir=log_dir)
model = CNN().to(device)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss();
#optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
#writer.add_graph(model)
writer.add_hparams(hparam_dict=train_spec,metric_dict={})

In [25]:
for epoch in range(1, epochs + 1):
    epoch_dat=train(epoch)
    test()
    #print('saved epoch '+str(epoch))
    #scheduler.step()
writer.flush()
writer.close()

Train Epoch: 1 [8000/100000 (8%)]	Loss: 3.234474
Train Epoch: 1 [16000/100000 (16%)]	Loss: 2.257017
Train Epoch: 1 [24000/100000 (24%)]	Loss: 1.431118
Train Epoch: 1 [32000/100000 (32%)]	Loss: 0.987215
Train Epoch: 1 [40000/100000 (40%)]	Loss: 0.535908
Train Epoch: 1 [48000/100000 (48%)]	Loss: 0.333240
Train Epoch: 1 [56000/100000 (56%)]	Loss: 0.257507
Train Epoch: 1 [64000/100000 (64%)]	Loss: 0.188729
Train Epoch: 1 [72000/100000 (72%)]	Loss: 0.210575
Train Epoch: 1 [80000/100000 (80%)]	Loss: 0.132511
Train Epoch: 1 [88000/100000 (88%)]	Loss: 0.116477
Train Epoch: 1 [96000/100000 (96%)]	Loss: 0.075812

Test set: Average loss: -2.9302, Accuracy: 99897/100000 (100%)

Train Epoch: 2 [8000/100000 (8%)]	Loss: 0.094199
Train Epoch: 2 [16000/100000 (16%)]	Loss: 0.060795
Train Epoch: 2 [24000/100000 (24%)]	Loss: 0.063859
Train Epoch: 2 [32000/100000 (32%)]	Loss: 0.067950
Train Epoch: 2 [40000/100000 (40%)]	Loss: 0.031534
Train Epoch: 2 [48000/100000 (48%)]	Loss: 0.029613
Train Epoch: 2 [56000

KeyboardInterrupt: 

In [ ]:
MNIST_train_data = torchvision.datasets.FashionMNIST('/om/user/ehoseini/MyData/mnist_data/',train=True,download=True,transform = transforms.Compose([
        transforms.ToTensor()]))
MNIST_test_data = torchvision.datasets.FashionMNIST('/om/user/ehoseini/MyData/mnist_data/',train=False,download=True,transform = transforms.Compose([
        transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(
    dataset=MNIST_train_data,
    batch_size=124,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    dataset=MNIST_test_data,
    batch_size=1000,
    shuffle=False)


class Network(nn.Module):
    def __init__(self):
        super().__init__()

        # define layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # conv 1
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # conv 2
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # fc1
        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)

        # fc2
        t = self.fc2(t)
        t = F.relu(t)

        # output
        t = self.out(t)
        # don't need softmax here since we'll use cross-entropy as activation.

        return t